In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread
from skimage.color import rgb2gray
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Mount Google Drive
drive.mount('/content/drive')

# SVD for Compression
def svd_reduction(images, k, target_shape=(64, 64)):
    """
    Applies SVD to reduce the dimensionality of images.

    Parameters:
        images (list of ndarray): List of images as 2D arrays.
        k (int): Number of singular values to keep.
        target_shape (tuple): Target shape for resizing images.

    Returns:
        ndarray: Reduced feature matrix.
    """
    features = []
    for image in images:
        # Remove the alpha channel if it exists
        if image.shape[2] == 4:
            image = image[:, :, :3]
        # Convert to grayscale if necessary
        if image.ndim == 3:
            image = rgb2gray(image)
        # Resize image to the target shape
        image = resize(image, target_shape, anti_aliasing=True)
        # Apply SVD
        U, S, VT = np.linalg.svd(image, full_matrices=False)
        # Keep the first k singular values/vectors
        reduced_features = np.dot(U[:, :k], np.diag(S[:k]))
        features.append(reduced_features.flatten())  # Flatten the reduced matrix
    return np.array(features)

# Softmax Function
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Classification Pipeline
def classify(images, labels, k, learning_rate, num_iterations):
    """
    Pipeline to classify images using SVD and Gradient Descent.

    Parameters:
        images (list of ndarray): List of input images.
        labels (list): Corresponding labels.
        k (int): Number of singular values to keep in SVD.
        learning_rate (float): Learning rate for Gradient Descent.
        num_iterations (int): Number of iterations for Gradient Descent.
    """
    # Step 1: SVD Reduction
    print("Reducing dimensions with SVD...")
    X = svd_reduction(images, k)
    X = X.reshape(X.shape[0], -1)  # Flatten the reduced features

    # Step 2: One-Hot Encode Labels
    encoder = OneHotEncoder(sparse_output=False)
    y = encoder.fit_transform(np.array(labels).reshape(-1, 1))

    # Step 3: Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=labels)
    grid_search.fit(X_train, y_train)

    # Step 4:
    classifier = SVC()

    parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 10, 100, 1000]}]

    grid_search = GridSearchCV(classifier, parameters)
    grid_search.fit(X_train, y_train)

    # Test performance
    best_estimator = grid_search.best_estimator_
    y_prediction = best_estimator.predict(X_test)

    score = accuracy_score(y_test, y_prediction)
    print('f{}% of samples were correctly classified'.format(str(score * 100)))

    # Plot loss history
    plt.plot(loss_history)
    plt.title("Loss Over Iterations")
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.show()

    # Step 5: Evaluate Model
    logits = np.dot(X_test, weights)
    probabilities = softmax(logits)
    predictions = np.argmax(probabilities, axis=1)
    y_true = np.argmax(y_test, axis=1)
    accuracy = np.mean(predictions == y_true)
    print(f"Test Accuracy: {accuracy:.2f}")

    return weights, encoder

# Test Model with New Images
def test_model(new_images, weights, k, target_shape=(64, 64), label_mapping=None):
    """
    Tests the trained model on new images.

    Parameters:
        new_images (list of ndarray): List of new images to test.
        weights (ndarray): Trained weights of the model.
        k (int): Number of singular values used during training.
        target_shape (tuple): Target shape for resizing images.
        label_mapping (list): List of original labels corresponding to class indices.

    Returns:
        list: Predicted labels for the new images.
    """
    # Reduce dimensions of new images using SVD
    print("Processing new images with SVD...")
    X_new = svd_reduction(new_images, k, target_shape)
    X_new = X_new.reshape(X_new.shape[0], -1)  # Flatten the features

    # Compute logits and probabilities
    logits = np.dot(X_new, weights)
    probabilities = softmax(logits)

    # Get predicted indices
    predictions = np.argmax(probabilities, axis=1)

    # Map indices to labels
    if label_mapping is not None:
        predictions = [label_mapping[idx] for idx in predictions]

    return predictions

# Importing images for training
images = []
labels = []

# Load Nazli images (Nazli (1-30))
for i in range(1, 80):
    image = imread(f'/content/drive/My Drive/Grapevine_Leaves_Image_Dataset/Nazli/Nazli ({i}).png')
    images.append(image)
    labels.append("Nazli")

# Load Dimnit images (Dimnit (1-30))
for i in range(1, 80):
    image = imread(f'/content/drive/My Drive/Grapevine_Leaves_Image_Dataset/Dimnit/Dimnit ({i}).png')
    images.append(image)
    labels.append("Dimnit")

# Load Buzgulu images (Buzgulu (1-30))
for i in range(1, 80):
    image = imread(f'/content/drive/My Drive/Grapevine_Leaves_Image_Dataset/Buzgulu/Buzgulu ({i}).png')
    images.append(image)
    labels.append("Buzgulu")

images = np.array(images)
labels = np.array(labels)

# Parameters
k = 50  # Number of singular values
learning_rate = 0.1
num_iterations = 25000

# Train the model
weights, encoder = classify(images, labels, k, learning_rate, num_iterations)

# Test with new images
new_images = []
main_labels = ["Nazli"]
for i in range(90, 100):
    for j in (main_labels):
      new_image = imread(f'/content/drive/My Drive/Grapevine_Leaves_Image_Dataset/{j}/{j} ({i}).png')
      new_images.append(new_image)

if new_images:
    label_mapping = encoder.categories_[0]
    predicted_labels = test_model(new_images, weights, k, target_shape=(64, 64), label_mapping=label_mapping)

    print("\nPredicted labels for new images:")
    for i, label in enumerate(predicted_labels):
        print(f"Image {i + 1}: Predicted Label = {label}")
else:
    print("No new images found for testing.")

KeyboardInterrupt: 